<a href="https://colab.research.google.com/github/SupunGurusinghe/chatGPT-API-assignment-/blob/main/Innodata_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **First phase of the assignment**

**Install the OpenAI Python package to access the GPT-3 API**

In [2]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.0 MB/s eta 0:00:00


**Import the necessary modules in your Python script**

In [3]:
import openai
import json

**Authenticate with OpenAI using your API key**

In [4]:
openai.api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

**Define the text of the research paper you want to analyze**

In [ ]:
text = """Named Entity Recognition
March 2014
DOI: 10.1007/978-3-642-45358-8_7
In book: Natural Language Processing of Semitic Languages
Behrang Mohit"""

**Define the prompts to extract specific information from the text**

In [ ]:
title_prompt = "What is the title of the paper?"

**Use the OpenAI API to generate the text for each prompt**

In [ ]:
title_response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=title_prompt + "\nText: " + text,
    temperature=0.5,
    max_tokens=200,
    n = 1,
    stop=None,
    frequency_penalty=0,
    presence_penalty=0
)

**Parse the response from the OpenAI API and extract the relevant information**

In [ ]:
title = title_response.choices[0].text.strip()

**Print extracted information**

In [ ]:
print(title)

The title of the paper is "Named Entity Recognition."


**Write the extracted information to a JSON file**

In [ ]:
data = {
    "title": title,
    # "authors": authors,
    # Add additional fields as needed
}

with open("output.json", "w") as outfile:
    json.dump(data, outfile, indent=4)


## **Iterate the process**

In [ ]:
prompt = "What is the title of the paper. Who is the author."
prompt_list = prompt.split(". ")

data = {}

for prompt_txt in prompt_list:
  details_response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt_txt + "\nText: " + text,
    temperature=0.5,
    max_tokens=200,
    n = 1,
    stop=None,
    frequency_penalty=0,
    presence_penalty=0
  )  
  details = details_response.choices[0].text.strip()
  print(details)

  data[prompt_txt] = details


The title of the paper is "Named Entity Recognition."
Behrang Mohit is the author.


In [ ]:
with open("output.json", "w") as outfile:
    json.dump(data, outfile, indent=4)

# **Second phase of the assignment - for pdf characters < 4096**

In [13]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 5.4 MB/s eta 0:00:00


In [14]:
import PyPDF2
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving How_to_Summarize_a_Research_Article1.pdf to How_to_Summarize_a_Research_Article1.pdf


In [ ]:
for filename in uploaded.keys():
    with open(filename, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        text = ''
        for page in range(num_pages):
            page_obj = pdf_reader.pages[page]
            text += page_obj.extract_text()

        pdf_file.close()

In [ ]:
prompt = "What is the title of this text. Who is the author of this text. What are the sub topics in this text."
prompt_list = prompt.split(". ")

data = {}

for prompt_txt in prompt_list:
  details_response = openai.Completion.create(
    engine="text-davinci-002",
    prompt=prompt_txt + "\nText: " + text,
    temperature=0.5,
    max_tokens=200,
    n = 1,
    stop=None,
    frequency_penalty=0,
    presence_penalty=0
  )  
  details = details_response.choices[0].text.strip()
  print(details)

  data[prompt_txt] = details

Summarizing a Research Article
The author of the text is unknown.
The sub topics in this text are: 
- How to Summarize a Research Article 
- Determining Your Focus 
- Reading the Article 
- Writing the Summary 
- Editing for Completeness and Accuracy 
- Editing for Style


In [ ]:
with open("output.json", "w") as outfile:
    json.dump(data, outfile, indent=4)

# **Third phase of the assignment - for pdf characters > 4096**

In [5]:
import re

In [6]:
def pdf_to_text(uploaded):
  for filename in uploaded.keys():
      with open(filename, 'rb') as pdf_file:
          pdf_reader = PyPDF2.PdfReader(pdf_file)
          num_pages = len(pdf_reader.pages)

          text = ''
          for page in range(num_pages):
              page_obj = pdf_reader.pages[page]
              text += page_obj.extract_text()

          pdf_file.close()
  return text

In [79]:
# Split the long input text into smaller chunks that are each within the maximum context length
def split_by_smaller_chunks(text, MAX_TOKENS_PER_REQUEST = 4000):
  input_chunks = [text[i:i+2048] for i in range(0, len(text), MAX_TOKENS_PER_REQUEST)]
  return input_chunks

In [69]:
# Split the input text into sentences
def split_by_sentence(text, MAX_TOKENS_PER_REQUEST = 4000):
  sentences = re.split('(?<=\w\.) ', text)

  input_chunks = []
  chunk = ''
  for sentence in sentences:
      if len(chunk) + len(sentence) < MAX_TOKENS_PER_REQUEST:
          chunk += sentence
      else:
          input_chunks.append(chunk)
          chunk = sentence

  # Add the last chunk to the list
  if chunk:
      input_chunks.append(chunk)

  return input_chunks

In [70]:
# Split the input text into paragraphs
def split_by_paragraph(text, MAX_TOKENS_PER_REQUEST = 4000):
  paragraphs = text.split("\n\n")

  input_chunks = []
  chunk = ''
  for paragraph in paragraphs:
      if len(chunk) + len(paragraph) < MAX_TOKENS_PER_REQUEST:
          chunk += paragraph
      else:
          input_chunks.append(chunk)
          chunk = paragraph

  # Add the last chunk to the list
  if chunk:
      input_chunks.append(chunk)

  return input_chunks

In [73]:
def openai_response(context, max_output_length=200, temperature=0.7, n=1, frequency_penalty=0, presence_penalty=0):
  # Generate the output using the GPT-3 API
  response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=context,
      max_tokens=max_output_length,
      temperature=0.7,
      n = 1,
      stop=None,
      frequency_penalty=0,
      presence_penalty=0
  )
  return response

In [80]:
def generate_response(text, prompt, method, MAX_TOKENS_PER_REQUEST = 4000, max_output_length=200, temperature=0.7, n=1, frequency_penalty=0, presence_penalty=0):
  if method == "split_by_smaller_chunks":
    input_chunks = split_by_smaller_chunks(text, MAX_TOKENS_PER_REQUEST = 4000)
  elif method == "split_by_sentence":
    input_chunks = split_by_sentence(text, MAX_TOKENS_PER_REQUEST = 4000)
  elif method == "split_by_paragraph":
    input_chunks = split_by_paragraph(text, MAX_TOKENS_PER_REQUEST = 4000)
  else:
    input_chunks = split_by_sentence(text, MAX_TOKENS_PER_REQUEST = 4000)

  results = []

  # Loop through the input chunks and generate an output for each one
  for chunk in input_chunks:

    for prompt_txt in prompt:
      # Concatenate the input chunk and prompt into a single string
      context = prompt_txt + "\nText: " + chunk

      # Generate the output using the GPT-3 API
      response = openai_response(context, max_output_length, temperature, n, frequency_penalty, presence_penalty)

      results.append(response.choices[0].text.strip())
  return results


In [28]:
uploaded = files.upload()
text = pdf_to_text(uploaded)

Saving How_to_Summarize_a_Research_Article1.pdf to How_to_Summarize_a_Research_Article1.pdf


In [ ]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup

#@title **Add Text for Web Articles and Blogs**

url = "https://en.wikipedia.org/wiki/Medium_(website)" #@param {type:"string"}
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
text = soup.get_text()

In [81]:
#@title **Enter a Prompt List**
Prompt = ["This article is published by who?", "Who are the group members of this?"] #@param {type:"raw"}
method = "split_by_sentence" #@param ["split_by_smaller_chunks", "split_by_sentence", "split_by_paragraph"]
MAX_TOKENS_PER_REQUEST = 4000 #@param {type:"number"}

if type(Prompt) == list:
  results = generate_response(text, Prompt, method=split_by_sentence)
  print("Intermediate Results \n========================\n")
  for res in range(0, len(results)):
    print(results[res])
    
  # join outputs
  s = "\n"
  results = s.join(results)

  data = {}
  for prompt_txt in Prompt:
    # Concatenate the input chunk and prompt into a single string
    context = prompt_txt + "\nText: " + results

    # Generate the output using the GPT-3 API
    response = openai_response(context)
    data[prompt_txt] = response.choices[0].text.strip()

  print("\n\nFinal Results \n========================\n")
  for key, val in data.items():
    print(f"{key}: {val}")
else:
  print("Enter a python list to the prompt")

Intermediate Results 

1.1 Uses of Plastic 
Plastic products have proven to be a great help for humans in many ways. It is used for 
packaging of food , for making medical equipment , for carrying water , for making 
automobiles, for making toys and many other products . 
 
1.2 Plastic as a material is not a problem; but how we manage it is an issue  
Plastic is a very useful material and can be recycled and reused.But plastic is a non- 
biodegradable material. It takes hundreds of years to decompose and leaves behind toxic 
chemicals that pollute soil and water bodies.This means that the plastic that is disposed of 
remains in the environment for a long time if not managed properly. This is why plastic 
pollution has become a major environmental concern . Introduction to Plastic  Pollution   
2 

When plastic is not managed properly
1.1 Uses of Plastic  
Plastics are used in many types of products – from children’s toys to medical equipment. Some 
of the common uses of plastic are: 
•

In [ ]:
with open("results.json", "w") as outfile:
    json.dump(data, outfile, indent=4)

In [82]:
#@title **Change Response Parameters**
Prompt = ["This article is published by who?", "Who are the group members of this?"] #@param {type:"raw"}
method = "split_by_sentence" #@param ["split_by_smaller_chunks", "split_by_sentence", "split_by_paragraph"]
max_output_length = 200 #@param {type:"integer"}
temperature = 0.7 #@param {type:"number"}
n = 1 #@param {type:"integer"}
frequency_penalty = 0 #@param {type:"number"}
presence_penalty = 0 #@param {type:"number"}
MAX_TOKENS_PER_REQUEST = 4000 #@param {type:"integer"}

if type(Prompt) == list:
  results = generate_response(text, Prompt, method, MAX_TOKENS_PER_REQUEST, max_output_length, temperature, n, frequency_penalty, presence_penalty)
  print("Intermediate Results \n========================\n")
  for res in range(0, len(results)):
    print(results[res])
    
  # join outputs
  s = "\n"
  results = s.join(results)

  data = {}
  for prompt_txt in Prompt:
    # Concatenate the input chunk and prompt into a single string
    context = prompt_txt + "\nText: " + results

    # Generate the output using the GPT-3 API
    response = openai_response(context, max_output_length, temperature, n, frequency_penalty, presence_penalty)
    data[prompt_txt] = response.choices[0].text.strip()
    
  print("\n\nFinal Results \n========================\n")
  for key, val in data.items():
    print(f"{key}: {val}")
else:
  print("Enter a python list to the prompt")

Intermediate Results 

1.1 Uses of Plastic  
Plastic has great usability in day to day life, whether in the form of rigid or flexible material. 
It is used in packaging of food, medicines and consumer products. Plastic is also used in 
manufacturing of cars, boats, machines and other items. It is also used for making medical 
equipments, toys and electrical fittings. Plastic is used in construction of buildings and 
infrastructure, furniture and even in agricultural processes.  
1.2 Plastic as a Material is not a Problem; but How We Manage it is an Issue  
Plastic is an incredibly useful material; however,  the way we manage it has caused the world 
a great deal of harm. It is estimated that 8 million tons of plastics waste enter the oceans every 
year. This plastic waste is killing fish, birds and marine animals, and it is contaminating the 
Introduction to Plastic Pollution
1.1  Uses of Plastic 
Plastics are used in almost all the products that we use and come in contact with. 
 
Som

In [ ]:
with open("results.json", "w") as outfile:
    json.dump(data, outfile, indent=4)

# Experiment

In [83]:
def generate_response_2(text, prompt, keywords, method, MAX_TOKENS_PER_REQUEST=4000, max_output_length=200, temperature=0.7, n=1, frequency_penalty=0, presence_penalty=0):
  if method == "split_by_smaller_chunks":
    input_chunks = split_by_smaller_chunks(text, MAX_TOKENS_PER_REQUEST=4000)
  elif method == "split_by_sentence":
    input_chunks = split_by_sentence(text, MAX_TOKENS_PER_REQUEST=4000)
  elif method == "split_by_paragraph":
    input_chunks = split_by_paragraph(text, MAX_TOKENS_PER_REQUEST=4000)
  else:
    input_chunks = split_by_sentence(text, MAX_TOKENS_PER_REQUEST=4000)

  chunk_count = 0
  results = []

  summary_count = (4096 - len(prompt)) / len(input_chunks)

  # Loop through the input chunks and generate an output for each one
  for chunk in input_chunks:
    chunk_count += 1

    # Concatenate the input chunk and prompt into a single string
    context = f"{chunk} \n\n Shorten this text by including main points of text having keywords {keywords} and word count less than or equal to a word count of {summary_count}"
    
    # Generate the output using the GPT-3 API
    response = openai_response(context, max_output_length, temperature, n, frequency_penalty, presence_penalty)

    results.append(response.choices[0].text.strip())
  return results

In [49]:
uploaded = files.upload()
text = pdf_to_text(uploaded)

Saving Annexure_B6_ Braille-Booklet-on-Plastic-Pollution.pdf to Annexure_B6_ Braille-Booklet-on-Plastic-Pollution.pdf


In [86]:
#@title Chunk by chunk
prompt = ["This article is published by who?", "Who are the group members of this?"] #@param {type:"raw"}
method = "split_by_sentence" #@param ["split_by_smaller_chunks", "split_by_sentence", "split_by_paragraph"]
MAX_TOKENS_PER_REQUEST = 4000 #@param {type:"number"}
j = " "
joined_prompt = j.join(prompt)
method = "split_by_sentence"
keywords_text = "Generate keywords of following text." + "\nText: " + joined_prompt
keywords = openai_response(keywords_text)

results = generate_response_2(text, prompt, keywords, method, MAX_TOKENS_PER_REQUEST)

# join outputs
s = "\n"
results = s.join(results)

data = {}
for p_text in prompt:
  # Concatenate the input chunk and prompt into a single string
  context = p_text + "\nText: " + results

  # Generate the output using the GPT-3 API
  response = openai_response(context)
  res = response.choices[0].text.strip()
  data[p_text] = res
for key, val in data.items():
  print(f"{key}: {val}")

This article is published by who?: This article is published by the Energy and Resources Institute (TERI) and sponsored by the United Nations Environment Programme (UNEP).
Who are the group members of this?: The group members for this publication are Dr. Anjali Parasnis, Ms. Vaishnavi Barthwal, Mr. Manish Asodekar, Ms. Pranali Chavan and Mr. Prakash Joshi.


In [75]:
with open("exp_results.json", "w") as outfile:
    json.dump(data, outfile, indent=4)